In [ ]:
import pandas as pd
import numpy as np
import requests
from urllib.parse import urlencode
import base64
import webbrowser
import json

#### Load Kaggle dataset of most streamed songs on Spotify, 2023

In [ ]:
df1 = pd.read_csv('spotify-2023.csv',encoding='latin-1')

In [ ]:
df1.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


#### Authorize Spotify API

In [ ]:
# Set credentials

client_id = 'e569e9ac27934e209405a9fa233794d4'
client_secret = 'd7c2dac1d2444f66841f184d4ec076f4'
redirect_uri = 'http://localhost:3300/callback'
endpoint = 'https://accounts.spotify.com/authorize'
header_vals = {'client_id':client_id,'response_type':'code','redirect_uri':redirect_uri}

webbrowser.open("https://accounts.spotify.com/authorize?" + urlencode(header_vals))

True

In [ ]:
code = 'AQB84Xula7tDr_Q7tuV21mEsO5vLk5vHJYRlxLbLti_jAqQwx05Ug2Ax0puDGg3kqlKK1wxmjoftSmXN4Ic-aCHQ44KKd_Y-xLDern-t_bxK1r59tkVqV4bB5870neMkvLsczDDLKp5CoMF9SUMtWRGnKGae_sGyPiMDRb7urmTUjA'
encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:3300/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
ACCESS_TOKEN = r.json()['access_token']

In [ ]:
user_headers = {
    "Authorization": "Bearer " + ACCESS_TOKEN,
    "Content-Type": "application/json"
}

#### Call Playlists endpoint to get top playlists + tracks

In [ ]:
# Download list of featured playlists + metadata

featured_playlists = requests.get("https://api.spotify.com/v1/browse/featured-playlists", headers=user_headers)
featured_playlists_json = featured_playlists.json()

with open("playlist_metadata.json", "w") as outfile:
    json.dump(featured_playlists_json, outfile)

In [ ]:
# Get playlist_ids to get the tracks on each playlist

playlist_ids = {}

for p in featured_playlists_json['playlists']['items']:
    playlist_ids[p['id']] = p['name']

In [ ]:
# Download list of tracks for each play list
playlist_tracks = []

for pid in playlist_ids:
    url = "https://api.spotify.com/v1/playlists/"+str(pid)+"/tracks"
    output = requests.get(url, headers=user_headers)
    json_object = output.json()
    json_object['playlist_id'] = pid
    playlist_tracks.append(json_object)

In [ ]:
# Write playlist tracks to json

with open("playlist_tracks.json", "w") as outfile:
    json.dump(playlist_tracks, outfile)